In [1]:
import tensorflow as tf
import tensorflow_text 
import numpy as np
from tqdm import tqdm
import pandas as pd

import timeit
import traceback
import contextlib
from tensorflow_io.bigquery import BigQueryClient
client = BigQueryClient()

## defining dataset

## defining models

## running inference

In [4]:
filenames = ['/home/jupyter/data/all_in.csv']
buffer_size = 64
batch_size = 64
record_defaults=[tf.string] * 3

ds = tf.data.experimental.CsvDataset(
    filenames, record_defaults, compression_type=None, buffer_size=buffer_size,
    header=False, field_delim=';', use_quote_delim=True, na_value='',
    select_cols=None
)

In [5]:
i = 0
for batch in ds.batch(batch_size).take(1):
    i += 1

In [46]:
@contextlib.contextmanager
def options(options):
    old_opts = tf.config.optimizer.get_experimental_options()
    tf.config.optimizer.set_experimental_options(options)
    try:
        yield
    finally:
        tf.config.optimizer.set_experimental_options(old_opts)

In [6]:
imported = tf.saved_model.load(saved_model_path, ["serve"])
def compute():
    @tf.function
    def simple_function(input_arg):
         return imported.signatures['serving_default'](input_arg[0])
    return simple_function

pred = compute()
  

In [59]:
with options({'loop_optimization': False, 'shape_optimization': False, 'remapping': False, 'debug_stripper': False,'disable_model_pruning': False}):
    print(tf.config.optimizer.get_experimental_options())
    #simple_function = test_function_1()
    # Trace once
    #x = tf.constant(2.2)
    #simple_function(x)
    print("Vanilla execution:", timeit.timeit(lambda: pred(batch), number = 1), "s")

{'constant_folding': False, 'shape_optimization': False, 'remapping': False, 'loop_optimization': False, 'debug_stripper': False, 'disable_model_pruning': False, 'disable_meta_optimizer': False}
Vanilla execution: 30.601016760999755 s


In [52]:
#pred(batch)
print("Vanilla execution:", timeit.timeit(lambda: pred(batch), number = 1), "s")

Vanilla execution: 20.414218624000114 s


In [60]:
a = pred(batch)

In [7]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='tpu3-antoine-us-central1-a')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

ValueError: Could not lookup TPU metadata from name 'tpu3-antoine-us-central1-a'. Please doublecheck the tpu argument in the TPUClusterResolver constructor. Exception: <HttpError 404 when requesting https://tpu.googleapis.com/v1/projects/for-antoine/locations/us-central1-a/nodes/tpu3-antoine-us-central1-a?alt=json returned "Resource 'projects/for-antoine/locations/us-central1-a/nodes/tpu3-antoine-us-central1-a' was not found". Details: "[{'@type': 'type.googleapis.com/google.rpc.ResourceInfo', 'resourceName': 'projects/for-antoine/locations/us-central1-a/nodes/tpu3-antoine-us-central1-a'}]">